In [100]:
import pandas as pd
import requests as req
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [103]:
def get_chart_image(item_name, url):
    res = req.get(url)
    soup = bs(res.text, 'html.parser')

    image = soup.select_one('div.chart > img')['src']
    # 기본 차트는 선차트이므로 봉차트로 변경한다
    image_url = image.replace('area', 'candle')
    
    with urlopen(image_url) as f:
        with open('./images/stock/' + item_name +'.jpg','wb') as h:
            data = f.read()
            h.write(data)

In [106]:
url = 'https://finance.naver.com/sise/sise_market_sum.naver?sosok=0'
res = req.get(url)

soup = bs(res.text, 'html.parser')

my_items = ['삼성전자', 'SK하이닉스', 'LG화학', '현대차', 'NAVER']  # 관심종목

df = pd.DataFrame(columns=['No', '종목명', '현재가', '전일비', '등락율', '액면가', '시가총액',
                           '상장주식수', '외국인비율', '거래량', 'PER', 'ROE', '토론실'])
items = soup.select('table.type_2 > tbody > tr')
for i in range(len(items)):
    a_tag = items[i].select_one('td > a.tltle')
    
    # for 루프를 돌면서 관심종목만 뽑아낸다
    if a_tag != None and a_tag.text in my_items:
        item_tag = a_tag.parent.parent
        columns = item_tag.select('td')
        
        data = [column.text.strip() for column in columns]
        # 전일비가 항상 양수로 나오므로 등락율이 -일 때는 -부호를 붙인다.
        if data[4].find('-') >= 0:
            data[3] = '-' + data[3]
        
        df = df.append(pd.Series(data, index=df.columns), ignore_index=True)
        print(data)
        
        chart_url = 'https://finance.naver.com/' + a_tag.get('href')
        get_chart_image(a_tag.text, chart_url)
        
print('크롤링이 끝났습니다.')

['1', '삼성전자', '68,500', '100', '+0.15%', '100', '4,089,301', '5,969,783', '52.18', '5,151,471', '10.34', '17.07', '']
['3', 'SK하이닉스', '97,200', '-700', '-0.72%', '5,000', '707,618', '728,002', '50.20', '1,397,279', '-30.38', '3.56', '']
['5', 'LG화학', '728,000', '9,000', '+1.25%', '5,000', '513,912', '70,592', '48.03', '118,512', '34.93', '6.95', '']
['8', '현대차', '207,000', '-500', '-0.24%', '5,000', '437,870', '211,532', '32.07', '198,749', '6.30', '9.36', '']
['10', 'NAVER', '203,500', '-8,500', '-4.01%', '100', '333,840', '164,049', '48.15', '880,053', '51.83', '3.29', '']
크롤링이 끝났습니다.


In [107]:
df

,No,종목명,현재가,전일비,등락율,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
0,1,삼성전자,"68,500",100,+0.15%,100,"4,089,301","5,969,783",52.18,"5,151,471",10.34,17.07,
1,3,SK하이닉스,"97,200",-700,-0.72%,"5,000","707,618","728,002",50.20,"1,397,279",-30.38,3.56,
2,5,LG화학,"728,000","9,000",+1.25%,"5,000","513,912","70,592",48.03,"118,512",34.93,6.95,
3,8,현대차,"207,000",-500,-0.24%,"5,000","437,870","211,532",32.07,"198,749",6.30,9.36,
4,10,NAVER,"203,500","-8,500",-4.01%,100,"333,840","164,049",48.15,"880,053",51.83,3.29,
